In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split



path = input('Por favor adicione o diretorio de trabalho?')
for dirname, _, filename in os.walk(path, topdown = True):
    for filename in filename:
        if filename == 'test.csv':
                teste = pd.read_csv(os.path.join(dirname,filename))
        else:
            if filename == 'train.csv':
                treino = pd.read_csv(os.path.join(dirname,filename))
            else:
                print("Deu ruim irmão!!")
            
X = treino.drop('Cover_Type',axis=1) 
y= treino.Cover_Type

Por favor adicione o diretorio de trabalho? C:\github\Kaggle_Challenges\ML_Competitions\data


# GMM

In [ ]:
#Testando quantidade de componentes
n_components = np.arange(1, 35)
models = [GaussianMixture(n, covariance_type='full',reg_covar=1, random_state=0).fit(teste)
          for n in n_components]

f, ax = plt.subplots(figsize=(12,8))
plt.plot(n_components, [m.aic(teste) for m in models], label='AIC')
plt.legend(loc='best')
plt.xlabel('n_components');

C:\Users\PC\Anaconda3\lib\site-packages\sklearn\mixture\base.py:265: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  % (init + 1), ConvergenceWarning)


In [ ]:
#Gerando os Grupos

gmix = GaussianMixture(n_components = 22)
gmix.fit(teste)

#Separando X e Y
X_mix = treino.drop('Cover_Type', axis =1)
y_mix = treino['Cover_Type']

treino['Cluster_teste'] = gmix.predict(X_mix)
teste['Cluster_teste'] = gmix.predict(teste)

# Gerando os novos datasets

treino.to_csv('train.csv')
teste.to_csv('test.csv')

# Analise das variáveis de distância.

In [ ]:
#Acrecentando variavel com a distancia euclideana entre as distancias para água.
def euc_dist(df):
    cols = ['Elevation', 'Horizontal_Distance_To_Hydrology','Horizontal_Distance_To_Fire_Points']
    df['Euclidean_Dist'] = np.sqrt((df.Vertical_Distance_To_Hydrology**2 + df.Horizontal_Distance_To_Hydrology**2))
    df['Dist_Meam'] = df[cols[0:]].mean(axis=1)
    df['Vertical_Diff'] = df['Elevation']-df['Vertical_Distance_To_Hydrology']
    return df

euc_dist(treino)
euc_dist(teste)

treino.to_csv('train.csv')
teste.to_csv('test.csv')

# Analise Elevation

In [ ]:
def elevation(df):
    #df['Elevationlog'] = np.log1p(df['Elevation'])
    #df['Elevation2'] = df['Elevation']**2
    df['ElevationEUC'] = df['Elevation']*df['Euclidean_Dist'] 
    df['ElevationV'] = df['Elevation']*df['Vertical_Distance_To_Hydrology']
    df['ElevationH'] = df['Elevation']*df['Horizontal_Distance_To_Hydrology']
    return df

elevation(treino)
elevation(teste)

treino.to_csv('train.csv')
teste.to_csv('test.csv')

# Aspect vs Slope

In [ ]:
def AspxSlop(df):
    df['AspxSlp'] = df['Aspect']*df['Slope']
    df['AspxSlp_Noon'] = df['Aspect']*df['Slope']*df['Hillshade_Noon']
    df['AspxSlp_3pm'] = df['Aspect']*df['Slope']*df['Hillshade_3pm']
    df['AspxSlp_9am'] = df['Aspect']*df['Slope']*df['Hillshade_9am']
    return df

AspxSlop(treino)
AspxSlop(teste)

treino.to_csv('train.csv')
teste.to_csv('test.csv')

# Analise das variaveis Shades

In [ ]:
def shade(df):
    SHADES = ['Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm']
    
    df['shade_noon_diff'] = df['Hillshade_9am'] - df['Hillshade_Noon']
    df['shade_3pm_diff'] = df['Hillshade_Noon'] - df['Hillshade_3pm']
    df['shade_all_diff'] = df['Hillshade_9am'] - df['Hillshade_3pm']
    df['shade_sum'] = df[SHADES].sum(axis=1)
    df['shade_mean'] = df[SHADES].mean(axis=1)
    return df


shade(treino)
shade(teste)

treino.to_csv('train.csv')
teste.to_csv('test.csv')

# Willderness

In [ ]:
#def will(df):
    

# Analise das variáveis de Solo

In [ ]:
# Somando as variavies de Solo
def solo_sum(df):
    cols=  ['Soil_Type1', 'Soil_Type2', 'Soil_Type3',
       'Soil_Type4', 'Soil_Type5', 'Soil_Type6', 'Soil_Type7', 'Soil_Type8',
       'Soil_Type9', 'Soil_Type10', 'Soil_Type11', 'Soil_Type12',
       'Soil_Type13', 'Soil_Type14', 'Soil_Type15', 'Soil_Type16',
       'Soil_Type17', 'Soil_Type18', 'Soil_Type19', 'Soil_Type20',
       'Soil_Type21', 'Soil_Type22', 'Soil_Type23', 'Soil_Type24',
       'Soil_Type25', 'Soil_Type26', 'Soil_Type27', 'Soil_Type28',
       'Soil_Type29', 'Soil_Type30', 'Soil_Type31', 'Soil_Type32',
       'Soil_Type33', 'Soil_Type34', 'Soil_Type35', 'Soil_Type36',
       'Soil_Type37', 'Soil_Type38', 'Soil_Type39', 'Soil_Type40']
    
    df['Soil_Sum'] = df[cols].sum(axis=1)
    return df

solo_sum(treino)
solo_sum(teste)

treino.to_csv('train.csv')
teste.to_csv('test.csv')

# Gerando dados artificias da classe 2